# Reader data for regional views
[T331359](https://phabricator.wikimedia.org/T331359)

## Unique Devices

In [2]:
from wmfdata import spark, hive

In [9]:
import wmfdata as wmf

In [15]:
unique_devices_query='''
SELECT
    DATE_FORMAT(month, 'YYYY-MM') AS month,
    SUM(unique_devices) AS unique_devices
FROM
(
    SELECT
      CONCAT(year,'-',LPAD(month,2,'0'),'-01') AS month,
      uniques_estimate as unique_devices
    FROM 
        wmf.unique_devices_per_project_family_monthly
    WHERE 
        year = 2023
      AND project_family = 'wikipedia'
) a

GROUP BY DATE_FORMAT(month, 'YYYY-MM') 
LIMIT 1000
'''

In [16]:
uniques=spark.run(unique_devices_query)

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [17]:
uniques

,month,unique_devices
0,2023-02,1556623574
1,2023-01,1623196487


In [ ]:
SELECT "project_family" AS "project_family",
       "country" AS "country",
       "country_code" AS "country_code",
       "uniques_underestimate" AS "uniques_underestimate",
       "uniques_offset" AS "uniques_offset",
       "uniques_estimate" AS "uniques_estimate",
       "year" AS "year",
       "month" AS "month"
FROM "wmf"."unique_devices_per_project_family_monthly"
WHERE "year" = 2023
  AND "month" = 2
LIMIT 100

In [22]:
inner_q= spark.run(
'''
    SELECT
      CONCAT(year,'-',LPAD(month,2,'0'),'-01') AS month,
      cr.wmf_region AS region,
      uniques_estimate as unique_devices
    FROM 
        wmf.unique_devices_per_project_family_monthly ud
    JOIN gdi.country_meta_data cr 
    ON ud.country_code=cr.country_code_iso_2
    WHERE 
        year = 2023 AND month=2
      AND project_family = 'wikipedia'
      
'''      
)    

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [23]:
inner_q.tail()

,month,region,unique_devices
242,2023-02-01,Northern & Western Europe,5149549
243,2023-02-01,Northern & Western Europe,67743
244,2023-02-01,Sub-Saharan Africa,7478889
245,2023-02-01,South Asia,2148568
246,2023-02-01,Sub-Saharan Africa,195427


In [24]:
inner_q.unique_devices.sum()

1555713012

In [30]:
unique_devices_by_region_test_query='''
SELECT
    DATE_FORMAT(month, 'YYYY-MM') AS month,
    region,
    SUM(unique_devices) AS unique_devices
FROM
(
        SELECT
      CONCAT(year,'-',LPAD(month,2,'0'),'-01') AS month,
      cr.wmf_region AS region,
      uniques_estimate as unique_devices
    FROM 
        wmf.unique_devices_per_project_family_monthly ud
    LEFT JOIN gdi.country_meta_data cr 
    ON ud.country_code=cr.country_code_iso_2
    WHERE 
        year = 2023 AND month=2
      AND project_family = 'wikipedia'
) a

GROUP BY DATE_FORMAT(month, 'YYYY-MM') , region

LIMIT 1000
'''

In [31]:
by_region_test=spark.run(unique_devices_by_region_test_query)

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [32]:
by_region_test

,month,region,unique_devices
0,2023-02,Latin America & Caribbean,138202924
1,2023-02,North America,301686319
2,2023-02,Northern & Western Europe,365700194
3,2023-02,Sub-Saharan Africa,38809852
4,2023-02,Middle East & North Africa,83056835
5,2023-02,Central & Eastern Europe & Central Asia,230458692
6,2023-02,None,910562
7,2023-02,"East, Southeast Asia, & Pacific",251929465
8,2023-02,South Asia,145868731
9,2023-02,UNCLASSED,0


In [39]:
country_region=spark.run('''
SELECT distinct country_code, country, country_code_iso_2, 
canonical_country_name, wmf_region
FROM wmf.unique_devices_per_project_family_monthly ud
--JOIN gdi.country_meta_data cr 
--ON ud.country_code!=cr.country_code_iso_2
--, gdi.country_meta_data cr 
--WHERE ud.country_code!=cr.country_code_iso_2
''')

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [48]:
country_region=spark.run('''
SELECT distinct country_code, country
FROM 
wmf.unique_devices_per_project_family_monthly ud
--gdi.country_meta_data cr
WHERE
country_code NOT IN 
--NOT EXISTS
(
SELECT distinct country_code
--, country
--, country_code_iso_2, canonical_country_name, wmf_region
FROM wmf.unique_devices_per_project_family_monthly ud
JOIN gdi.country_meta_data cr 
ON ud.country_code=cr.country_code_iso_2
)

--JOIN gdi.country_meta_data cr 
--ON ud.country_code!=cr.country_code_iso_2
--, gdi.country_meta_data cr 
--WHERE ud.country_code!=cr.country_code_iso_2
''')

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [49]:
country_region

,country_code,country
0,--,Unknown


In [50]:
inner_q1= spark.run(
'''
    SELECT
      CONCAT(year,'-',LPAD(month,2,'0'),'-01') AS month,
      country,
      uniques_estimate as unique_devices
    FROM 
        wmf.unique_devices_per_project_family_monthly ud
    WHERE 
        year = 2023 AND month=2
      AND project_family = 'wikipedia'
      AND country='Unknown'
      
'''      
)    

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [51]:
inner_q1

,month,country,unique_devices
0,2023-02-01,Unknown,910562


Caveat: there are some unique devices that are from `Unknown` location in the unique_devices table which is causing mismatch in the totals, i.e. the total count of unique devices by region will be lesser than the overall count of unique devices. 


In [52]:
unique_devices_by_region_query='''
SELECT
    DATE_FORMAT(month, 'YYYY-MM') AS month,
    region,
    SUM(unique_devices) AS unique_devices
FROM
(
        SELECT
      CONCAT(year,'-',LPAD(month,2,'0'),'-01') AS month,
      cr.wmf_region AS region,
      uniques_estimate as unique_devices
    FROM 
        wmf.unique_devices_per_project_family_monthly ud
    LEFT JOIN gdi.country_meta_data cr 
    ON ud.country_code=cr.country_code_iso_2
    WHERE 
        year >= 2018
      AND project_family = 'wikipedia'
) a

GROUP BY DATE_FORMAT(month, 'YYYY-MM') , region

LIMIT 1000
'''

In [53]:
unique_devices_by_region=spark.run(unique_devices_by_region_query)

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [54]:
unique_devices_by_region.head()

,month,region,unique_devices
0,2021-08,Sub-Saharan Africa,36496622
1,2020-11,Northern & Western Europe,409568201
2,2021-05,Middle East & North Africa,94105567
3,2018-11,Sub-Saharan Africa,27024177
4,2020-06,Sub-Saharan Africa,30096918


In [55]:
unique_devices_by_region.tail()

,month,region,unique_devices
604,2021-07,Sub-Saharan Africa,33906350
605,2019-05,Northern & Western Europe,395909136
606,2022-10,North America,316361218
607,2021-11,South Asia,142930372
608,2019-01,North America,307623845


In [56]:
unique_devices_by_region.to_csv('unique_devices_by_region')

In [62]:
#unique_devices_by_region.loc['region'].sum()
#unique_devices_by_region['Sum'] = unique_devices_by_region.loc['r2':'r4',['None']].sum(axis = 1)
unique_devices_by_region['Sum_Result'] = unique_devices_by_region.loc[0 : 1,["unique_devices" ]].sum(axis = 1)
#print"\nSumming some rows...\n",dataFrame
unique_devices_by_region

,month,region,unique_devices,Sum_Result
0,2021-08,Sub-Saharan Africa,36496622,36496622.0
1,2020-11,Northern & Western Europe,409568201,409568201.0
2,2021-05,Middle East & North Africa,94105567,NaN
3,2018-11,Sub-Saharan Africa,27024177,NaN
4,2020-06,Sub-Saharan Africa,30096918,NaN
...,...,...,...,...
604,2021-07,Sub-Saharan Africa,33906350,NaN
605,2019-05,Northern & Western Europe,395909136,NaN
606,2022-10,North America,316361218,NaN
607,2021-11,South Asia,142930372,NaN


# March 2023 data

In [5]:
unique_devices_by_region_query_march='''
SELECT
    CONCAT(year,'-',LPAD(month,2,'0')) AS month,
    region,
    SUM(unique_devices) AS unique_devices
FROM
(
        SELECT
      year, month,
      cr.wmf_region AS region,
      uniques_estimate as unique_devices
    FROM 
        wmf.unique_devices_per_project_family_monthly ud
    LEFT JOIN gdi.country_meta_data cr 
    ON ud.country_code=cr.country_code_iso_2
    WHERE 
        year = 2023
      AND month = 3
      AND project_family = 'wikipedia'
) a

GROUP BY CONCAT(year,'-',LPAD(month,2,'0')) , region

LIMIT 1000
'''

In [6]:
unique_devices_by_region_march=spark.run(unique_devices_by_region_query_march)

In [7]:
unique_devices_by_region_march

,month,region,unique_devices
0,2023-03,None,646816
1,2023-03,North America,325350995
2,2023-03,UNCLASSED,4
3,2023-03,South Asia,154447390
4,2023-03,Northern & Western Europe,391785868
5,2023-03,Central & Eastern Europe & Central Asia,235470488
6,2023-03,Middle East & North Africa,83903820
7,2023-03,"East, Southeast Asia, & Pacific",271976970
8,2023-03,Latin America & Caribbean,162573682
9,2023-03,Sub-Saharan Africa,40210191


In [2]:
import wmfdata as wmf

# April-May 2023 data

In [5]:
unique_devices_by_region_query_aprilmay='''
SELECT
    CONCAT(year,'-',LPAD(month,2,'0')) AS month,
    region,
    SUM(unique_devices) AS unique_devices
FROM
(
        SELECT
      year, month,
      cr.wmf_region AS region,
      uniques_estimate as unique_devices
    FROM 
        wmf.unique_devices_per_project_family_monthly ud
    LEFT JOIN gdi.country_meta_data cr 
    ON ud.country_code=cr.country_code_iso_2
    WHERE 
        year = 2023
      AND month IN (4,5)
      AND project_family = 'wikipedia'
) a

GROUP BY CONCAT(year,'-',LPAD(month,2,'0')) , region

LIMIT 1000
'''

In [6]:
unique_devices_by_region_aprilmay=wmf.spark.run(unique_devices_by_region_query_aprilmay)

SPARK_HOME: /usr/lib/spark3
Using Hadoop client lib jars at 3.2.0, provided by Spark.
PYSPARK_PYTHON=/opt/conda-analytics/bin/python3


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/21 17:52:02 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/06/21 17:52:02 WARN Utils: Service 'sparkDriver' could not bind on port 12000. Attempting port 12001.
23/06/21 17:52:02 WARN Utils: Service 'sparkDriver' could not bind on port 12001. Attempting port 12002.
23/06/21 17:52:02 WARN Utils: Service 'sparkDriver' could not bind on port 12002. Attempting port 12003.
23/06/21 17:52:02 WARN Utils: Service 'sparkDriver' could not bind on port 12003. Attempting port 12004.
23/06/21 17:52:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/06/21 17:52:03 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/06/21 17:52:03 WARN Utils: Service 'SparkUI' could n

In [7]:
unique_devices_by_region_aprilmay

,month,region,unique_devices
0,2023-05,Central & Eastern Europe & Central Asia,216587984
1,2023-04,"East, Southeast Asia, & Pacific",265239446
2,2023-05,Northern & Western Europe,383424788
3,2023-05,Latin America & Caribbean,151401620
4,2023-04,None,441097
5,2023-04,South Asia,149344389
6,2023-05,Sub-Saharan Africa,38748599
7,2023-04,Northern & Western Europe,397299539
8,2023-04,Middle East & North Africa,75145197
9,2023-04,North America,326209093


## June 2023 unique devices

In [2]:
unique_devices_by_region_query_june='''
SELECT
    CONCAT(year,'-',LPAD(month,2,'0')) AS month,
    region,
    SUM(unique_devices) AS unique_devices
FROM
(
        SELECT
      year, month,
      cr.wmf_region AS region,
      uniques_estimate as unique_devices
    FROM 
        wmf.unique_devices_per_project_family_monthly ud
    LEFT JOIN gdi.country_meta_data cr 
    ON ud.country_code=cr.country_code_iso_2
    WHERE 
        year = 2023
      AND month = 6
      AND project_family = 'wikipedia'
) a

GROUP BY CONCAT(year,'-',LPAD(month,2,'0')) , region

LIMIT 1000
'''

In [3]:
unique_devices_by_region_june=wmf.spark.run(unique_devices_by_region_query_june)

SPARK_HOME: /usr/lib/spark3
Using Hadoop client lib jars at 3.2.0, provided by Spark.
PYSPARK_PYTHON=/opt/conda-analytics/bin/python3


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/06 19:25:57 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/07/06 19:25:57 WARN Utils: Service 'sparkDriver' could not bind on port 12000. Attempting port 12001.
23/07/06 19:25:57 WARN Utils: Service 'sparkDriver' could not bind on port 12001. Attempting port 12002.
23/07/06 19:25:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/07/06 19:25:58 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/07/06 19:26:06 WARN Utils: Service 'org.apache.spark.network.netty.NettyBlockTransferService' could not bind on port 13000. Attempting port 13001.
23/07/06 19:26:06 WARN Utils: Service 'org.apache.spark.network.netty.NettyBlockTransferService' could not bind on 

In [4]:
unique_devices_by_region_june

,month,region,unique_devices
0,2023-06,Sub-Saharan Africa,35001074
1,2023-06,Latin America & Caribbean,140509275
2,2023-06,North America,309798736
3,2023-06,UNCLASSED,11
4,2023-06,"East, Southeast Asia, & Pacific",259969704
5,2023-06,None,387479
6,2023-06,Northern & Western Europe,362680899
7,2023-06,Central & Eastern Europe & Central Asia,194471817
8,2023-06,South Asia,152280065
9,2023-06,Middle East & North Africa,67810537


### July 2023 unique devices

In [2]:
unique_devices_by_region_query_july='''
SELECT
    CONCAT(year,'-',LPAD(month,2,'0')) AS month,
    region,
    SUM(unique_devices) AS unique_devices
FROM
(
        SELECT
      year, month,
      cr.wmf_region AS region,
      uniques_estimate as unique_devices
    FROM 
        wmf.unique_devices_per_project_family_monthly ud
    LEFT JOIN gdi.country_meta_data cr 
    ON ud.country_code=cr.country_code_iso_2
    WHERE 
        year = 2023
      AND month = 7
      AND project_family = 'wikipedia'
) a

GROUP BY CONCAT(year,'-',LPAD(month,2,'0')) , region

LIMIT 1000
'''

In [4]:
unique_devices_by_region_july=wmf.spark.run(unique_devices_by_region_query_july)

SPARK_HOME: /usr/lib/spark3
Using Hadoop client lib jars at 3.2.0, provided by Spark.
PYSPARK_PYTHON=/opt/conda-analytics/bin/python3


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/23 21:23:26 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/08/23 21:23:27 WARN Utils: Service 'sparkDriver' could not bind on port 12000. Attempting port 12001.
23/08/23 21:23:27 WARN Utils: Service 'sparkDriver' could not bind on port 12001. Attempting port 12002.
23/08/23 21:23:27 WARN Utils: Service 'sparkDriver' could not bind on port 12002. Attempting port 12003.
23/08/23 21:23:27 WARN Utils: Service 'sparkDriver' could not bind on port 12003. Attempting port 12004.
23/08/23 21:23:27 WARN Utils: Service 'sparkDriver' could not bind on port 12004. Attempting port 12005.
23/08/23 21:23:27 WARN Utils: Service 'sparkDriver' could not bind on port 12005. Attempting port 12006.
23/08/23 21:23:27 WARN Utils: Service 'spar

In [5]:
unique_devices_by_region_july

,month,region,unique_devices
0,2023-07,"East, Southeast Asia, & Pacific",264323273
1,2023-07,Middle East & North Africa,66050044
2,2023-07,Latin America & Caribbean,132371837
3,2023-07,None,408169
4,2023-07,Sub-Saharan Africa,35006407
5,2023-07,Northern & Western Europe,358773850
6,2023-07,UNCLASSED,25
7,2023-07,South Asia,160808140
8,2023-07,Central & Eastern Europe & Central Asia,187815052
9,2023-07,North America,309838511


In [1]:
import wmfdata as wmf

Looking at 2022-07 data to check for discrepancies since this 2022-07 is the first month for reliable YoY data. 
Context: When we did a YoY comparison of unique devices we found -8% drop which seemed high. When we looked at regional breakdown we saw increases in a lot of regions and unclassed category didn't account for the differences. Upon looking further we saw that 2022-07 unique devices calculated in 2022 vs now in 2023 was different. This was due to the data being fixed retroactively (for 2022-08 and 2022-09 snapshots) in unique_devices datasets due to the Overcount issue which was detected in August 2022 and resolved on Sep 2022. 

In [2]:
#2022-07
unique_devices_by_region_query_july2022='''
SELECT
    CONCAT(year,'-',LPAD(month,2,'0')) AS month,
    region,
    SUM(unique_devices) AS unique_devices
FROM
(
        SELECT
      year, month,
      cr.wmf_region AS region,
      uniques_estimate as unique_devices
    FROM 
        wmf.unique_devices_per_project_family_monthly ud
    LEFT JOIN gdi.country_meta_data cr 
    ON ud.country_code=cr.country_code_iso_2
    WHERE 
        year = 2022
      AND month = 7
      AND project_family = 'wikipedia'
) a

GROUP BY CONCAT(year,'-',LPAD(month,2,'0')) , region

LIMIT 1000
'''

In [3]:
unique_devices_by_region_july2022 = wmf.spark.run(unique_devices_by_region_query_july2022)

SPARK_HOME: /usr/lib/spark3
Using Hadoop client lib jars at 3.2.0, provided by Spark.
PYSPARK_PYTHON=/opt/conda-analytics/bin/python3


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/31 22:16:48 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/08/31 22:16:48 WARN Utils: Service 'sparkDriver' could not bind on port 12000. Attempting port 12001.
23/08/31 22:16:48 WARN Utils: Service 'sparkDriver' could not bind on port 12001. Attempting port 12002.
23/08/31 22:16:48 WARN Utils: Service 'sparkDriver' could not bind on port 12002. Attempting port 12003.
23/08/31 22:16:48 WARN Utils: Service 'sparkDriver' could not bind on port 12003. Attempting port 12004.
23/08/31 22:16:48 WARN Utils: Service 'sparkDriver' could not bind on port 12004. Attempting port 12005.
23/08/31 22:16:48 WARN Utils: Service 'sparkDriver' could not bind on port 12005. Attempting port 12006.
23/08/31 22:16:49 WARN Utils: Service 'Spar

In [4]:
unique_devices_by_region_july2022

,month,region,unique_devices
0,2022-07,"East, Southeast Asia, & Pacific",259861780
1,2022-07,North America,279497264
2,2022-07,None,1233016
3,2022-07,Middle East & North Africa,65591695
4,2022-07,Northern & Western Europe,359532391
5,2022-07,Latin America & Caribbean,139395666
6,2022-07,South Asia,155296135
7,2022-07,Central & Eastern Europe & Central Asia,197181979
8,2022-07,Sub-Saharan Africa,31412721


In [8]:
unique_devices_july2022= wmf.spark.run('''
SELECT
    CONCAT(year,'-',LPAD(month,2,'0'),'-01') AS month,
    SUM(unique_devices) AS unique_devices
FROM
(
    SELECT
       year, month,
      uniques_estimate as unique_devices
    FROM 
        wmf.unique_devices_per_project_family_monthly
    WHERE 
        year >= 2014
      AND project_family = 'wikipedia'
) a

WHERE  (year = 2022 AND month = 7)
GROUP BY CONCAT(year,'-',LPAD(month,2,'0'),'-01')
LIMIT 1000
'''
)

In [9]:
unique_devices_july2022

,month,unique_devices
0,2022-07-01,1489002647


In 2022-07 we had previously reported 1,648,975,154 unique devices. This is incorrect. The corrected unique devices for 2022-07 is 1,489,002,647

In [12]:
unique_devices_aug2022= wmf.spark.run('''
SELECT
    CONCAT(year,'-',LPAD(month,2,'0'),'-01') AS month,
    SUM(unique_devices) AS unique_devices
FROM
(
    SELECT
       year, month,
      uniques_estimate as unique_devices
    FROM 
        wmf.unique_devices_per_project_family_monthly
    WHERE 
        year >= 2014
      AND project_family = 'wikipedia'
) a

WHERE  (year = 2022 AND month > 6)
GROUP BY CONCAT(year,'-',LPAD(month,2,'0'),'-01')
LIMIT 1000
'''
)

In [15]:
unique_devices_aug2022.sort_values('month')

,month,unique_devices
5,2022-07-01,1489002647
3,2022-08-01,1542342588
0,2022-09-01,1554616211
2,2022-10-01,1683358474
1,2022-11-01,1602482850
4,2022-12-01,1580336784


This was part of testing done to verify the difference in unique devices between July 2022 and Aug 2022 vs July-Aug 2023. 
The data for July-Aug 2022 was fixed in September 2022 due to the [Unique Devices By Family Overcount](https://wikitech.wikimedia.org/wiki/Analytics/Data_Lake/Data_Issues/2021-02-09_Unique_Devices_By_Family_Overcount) issue

In 2022-08 we had previously reported 1,665,646,489 unique devices. This is incorrect. The corrected unique devices for 2022-08 is 1,542,342,588.

### August 2023 unique devices

In [16]:
unique_devices_by_region_query_aug='''
SELECT
    CONCAT(year,'-',LPAD(month,2,'0')) AS month,
    region,
    SUM(unique_devices) AS unique_devices
FROM
(
        SELECT
      year, month,
      cr.wmf_region AS region,
      uniques_estimate as unique_devices
    FROM 
        wmf.unique_devices_per_project_family_monthly ud
    LEFT JOIN gdi.country_meta_data cr 
    ON ud.country_code=cr.country_code_iso_2
    WHERE 
        year = 2023
      AND month = 8
      AND project_family = 'wikipedia'
) a

GROUP BY CONCAT(year,'-',LPAD(month,2,'0')) , region

LIMIT 1000
'''

In [18]:
unique_devices_by_region_aug=wmf.spark.run(unique_devices_by_region_query_aug)

In [19]:
unique_devices_by_region_aug

,month,region,unique_devices
0,2023-08,"East, Southeast Asia, & Pacific",272486471
1,2023-08,Middle East & North Africa,67049240
2,2023-08,North America,309218036
3,2023-08,Latin America & Caribbean,138582748
4,2023-08,Central & Eastern Europe & Central Asia,188744285
5,2023-08,South Asia,161663600
6,2023-08,Sub-Saharan Africa,35617019
7,2023-08,UNCLASSED,33
8,2023-08,None,418008
9,2023-08,Northern & Western Europe,355186148
